## User Based Collaborative Filtering

In [1]:
import numpy as np
import pandas as pd 

In [2]:
r_cols = ['userId', 'movieId', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols,
                      encoding='latin-1')
m_cols = ['movieId', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

In [3]:
def replace_name(x):
    return movies[movies['movieId']==x].title.values[0]
ratings.movieId = ratings.movieId.map(replace_name)

In [4]:
user_ratings = ratings.pivot_table(index = ['userId'], columns = ['movieId'], values = 'rating', fill_value=0.0)

In [5]:
random_row = np.random.choice(user_ratings.index,1)
test_user = user_ratings.loc[random_row[0]].tolist()
test_user = pd.Series(test_user, index=user_ratings.columns)

In [6]:
user_ratings.drop(random_row[0],axis=0,inplace=True)

In [7]:
#predict ratings on test_user's items that topN similar users have rated
global recommendation
recommendation=[]
def find_recommendation(N_neighbours):
    sm = 0
    cnt=0
    for i,y in enumerate(test_user):
        if y==0.0:
            for k in N_neighbours:     
                if user_ratings[test_user.index[i]][k[0]]>0.0:
                    sm += user_ratings[test_user.index[i]][k[0]]*k[1]
                    cnt+= k[1]
            if cnt==0.0:
                continue 
            recommendation.append((test_user.index[i],round(sm/cnt,3)))
            sm=0
            cnt=0

In [8]:
#selecting topN similar users
test_user_similarity = user_ratings.corrwith(test_user, axis = 1)
neighbours = []
N=10
for i,y in enumerate(test_user_similarity):
    if y>0.0:
        neighbours.append((test_user_similarity.index[i], y))      

neighbours.sort(reverse=True, key=lambda x:x[1])
find_recommendation(neighbours[:N])
recommendation.sort(key=lambda x:x[1],reverse=True)

In [9]:
print("Recommendations for User {}:".format(random_row[0]))
for i in recommendation[:N]:
    print("\n")
    print(i[0])

Recommendations for User 159:


Aliens (1986)


Clear and Present Danger (1994)


E.T. the Extra-Terrestrial (1982)


Gattaca (1997)


Georgia (1995)


Hercules (1997)


L.A. Confidential (1997)


Maximum Risk (1996)


Michael Collins (1996)


Othello (1995)
